In [1]:
#using Pkg;Pkg.add("LibPQ")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
 Installed CEnum ─────────────── v0.1.2
 Installed LayerDicts ────────── v1.0.0
 Installed Decimals ──────────── v0.4.0
 Installed Syslogs ───────────── v0.3.0
 Installed DocStringExtensions ─ v0.8.0
 Installed Memento ───────────── v0.12.1
 Installed LibPQ ─────────────── v0.11.2
  Updating `~/.julia/environments/v1.2/Project.toml`
  [194296ae] + LibPQ v0.11.2
  Updating `~/.julia/environments/v1.2/Manifest.toml`
  [fa961155] + CEnum v0.1.2
  [abce61dc] + Decimals v0.4.0
  [ffbed154] + DocStringExtensions v0.8.0
  [6f188dcb] + LayerDicts v1.0.0
  [194296ae] + LibPQ v0.11.2
  [f28f55f0] + Memento v0.12.1
  [cea106d9] + Syslogs v0.3.0
  Building LibPQ → `~/.julia/packages/LibPQ/7GUmX/deps/build.log`


In [2]:
using DataFrames
using LibPQ

┌ Info: Precompiling LibPQ [194296ae-ab2e-5f79-8cd4-7183a0a5a0d1]
└ @ Base loading.jl:1242


In [15]:
conn = LibPQ.Connection("host=localhost dbname=postgres user=nakada password=type_password_here")


PostgreSQL connection (CONNECTION_OK) with parameters:
  user = nakada
  password = ********************
  dbname = postgres
  host = 10.0.21.207
  port = 5432
  client_encoding = UTF8
  application_name = LibPQ.jl
  sslmode = prefer
  sslcompression = 1
  krbsrvname = postgres
  target_session_attrs = any

In [7]:
LibPQ.execute(conn, "CREATE TABLE mytable 
    (col1 SERIAL PRIMARY KEY NOT NULL, col2 VARCHAR(50), col3 INT)")

PostgreSQL result

In [8]:
st = LibPQ.prepare(conn, "INSERT INTO MYTABLE(col2, col3) VALUES (\$1,\$2)")

PostgreSQL prepared statement named __libpq_stmt_0__ with query INSERT INTO MYTABLE(col2, col3) VALUES ($1,$2)

In [9]:
LibPQ.execute(st,["testdata",7])

PostgreSQL result

In [10]:
LibPQ.execute(st,["testdata2",8])

PostgreSQL result

In [11]:
df = DataFrame(LibPQ.execute(conn, "SELECT * FROM mytable"))

,col1,col2,col3
,Int32⍰,String⍰,Int32⍰
1,1,testdata,7
2,2,testdata2,8


In [12]:
res = LibPQ.execute(conn, "SELECT * FROM mytable");

function print_res(res)
    cols = LibPQ.column_names(res)
    println(join(cols,"|"))
    for r in res
        println(join(getindex.(Ref(r),1:length(cols)), "|") )
    end
end
print_res(res)

col1|col2|col3
1|testdata|7
2|testdata2|8


In [13]:
LibPQ.execute(conn, "DROP TABLE mytable")
LibPQ.close(conn)